In [14]:
import pandas as pd # queried_data processing
import pandas_profiling as pp
import numpy as np # linear algebra

In [15]:
import pprint #indet json 
import requests #make http requests
import json
from qwikidata.sparql  import return_sparql_query_results #return sparql results
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl
from http.client import IncompleteRead
import time
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

In [16]:
data = pd.read_csv('final_dataset.csv')
data.dropna(subset=['imdb_id'], inplace=True)
data

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,...,languages,director_gender,distributor,imDbRating,imDbRatingVotes,rottenTomatoes,budget,gross,keywords,awards
5,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",USA,"September 24, 2021",2021.0,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",...,English,male,Netflix,7.7,110057.0,NaN,NaN,NaN,"island,vampire,community,secret,tv mini series","Nominated for 1 Primetime Emmy, 4 wins & 29 no..."
7,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993.0,TV-MA,125 min,"Dramas, Independent Movies, International Movies",...,"Akan, English",male,NaN,7.0,704.0,8.2/10,"$1,000,000 (estimated)","$2,198,984","west indies,model,fashion,plantation,africa","Awards, 1 nomination"
8,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021.0,TV-14,9 Seasons,"British TV Shows, Reality TV",...,English,NaN,BBC One,8.6,10022.0,NaN,NaN,NaN,"baking competition,baking bread,baker,food tas...","Won 2 BAFTA Awards, 10 wins & 21 nominations t..."
9,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021.0,PG-13,104 min,"Comedies, Dramas",...,"English, Spanish",male,Netflix,6.3,12944.0,4.2/10,NaN,NaN,"bird,mother,father,home,starling","Awards, 1 win"
10,TV Show,"Vendetta: Truth, Lies and The Mafia",NaN,"Pino Maniaci, Silvana Saguto, Bartolomeo Parrino","UK, Italy","September 24, 2021",2021.0,TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...",...,"English, Italian",NaN,NaN,6.7,278.0,NaN,NaN,NaN,tv mini series,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18283,TV Show,Terrahawks,Desmond Saunders,"DENISE BRYER, WINDSOR DAVIES, JEREMY HITCHEN, ...",UK,0,1983.0,ALL,3 Seasons,TV Shows,...,English,male,ITV,7.1,748.0,NaN,NaN,NaN,"timeframe 2020s,alien invasion sci fi,aunt nep...","Awards, 2 nominations"
18284,Movie,River,Emily Skye,"Mary Cameron Rogers, Alexandra Rose, Rob Marsh...",NaN,0,2021.0,16+,93 min,"Drama, Science Fiction, Suspense",...,NaN,female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18287,Movie,Outpost,Steve Barker,"Ray Stevenson, Julian Wadham, Richard Brake, M...",UK,0,2008.0,R,90 min,Action,...,English,male,Netflix,5.8,17347.0,NaN,"$300,000 (estimated)","$463,377","nazi zombie,mercenary,ex soldier,bar,nazi",NaN
18288,TV Show,Maradona: Blessed Dream,NaN,"Esteban Recagno, Ezequiel Stremiz, Luciano Vit...","USA, Argentina",0,2021.0,TV-MA,1 Season,"Drama, Sports",...,"Spanish, Italian, English",NaN,Amazon Prime Video,7.1,1875.0,NaN,NaN,NaN,"football,male nudity,athlete,female nudity,futbol",NaN


In [20]:
data.columns

Index(['type', 'title', 'director', 'cast', 'country', 'date_added_netflix',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'netflix', 'amazon', 'date_added_amazon', 'imdb_id', 'languages',
       'director_gender', 'distributor', 'imDbRating', 'imDbRatingVotes',
       'rottenTomatoes', 'budget', 'gross', 'keywords', 'awards'],
      dtype='object')

In [17]:
for i in data.columns:
    null_rate = data[i].isna().sum() / len(data) * 100 
    if null_rate > 0 :
        print("{} null rate: {}%".format(i,round(null_rate,2)))

director null rate: 15.91%
cast null rate: 0.23%
country null rate: 13.76%
rating null rate: 0.01%
languages null rate: 24.02%
director_gender null rate: 27.58%
distributor null rate: 26.18%
imDbRating null rate: 22.88%
imDbRatingVotes null rate: 22.88%
rottenTomatoes null rate: 62.58%
budget null rate: 78.98%
gross null rate: 68.45%
keywords null rate: 27.63%
awards null rate: 50.55%


In [18]:
set(data['imDbRatingVotes'].to_list())

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 9.0,
 nan,
 nan,
 12.0,
 nan,
 nan,
 7.0,
 8.0,
 17.0,
 18.0,
 19.0,
 20.0,
 14.0,
 22.0,
 23.0,
 24.0,
 16.0,
 26.0,
 27.0,
 5.0,
 29.0,
 30.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 33.0,
 37.0,
 39.0,
 38.0,
 32.0,
 42.0,
 43.0,
 44.0,
 45.0,
 40.0,
 47.0,
 48.0,
 41.0,
 46.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 10.0,
 58.0,
 60.0,
 59.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 163911.0,
 72.0,
 71.0,
 74.0,
 75.0,
 76.0,
 589899.0,
 15.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 196739.0,
 131.0,
 132.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
